In [1]:
import pandas as pd
import numpy as np

In [2]:
DATADIR = "/scratch2/hugo/ERA5"
CLIMSTOR = "/mnt/climstor/ecmwf/era5/raw"
DATERANGEPL = pd.date_range("19590101", "20211231")
DATERANGEML = pd.date_range("19770101", "20211231")

In [3]:
import cdsapi

c = cdsapi.Client()
for year in np.unique(DATERANGEPL.year)[1:]:
    c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'variable': 'u_component_of_wind',
            'pressure_level': [
                '700', '775', '850', '925',
            ],
            'year': str(year),
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '06:00', '12:00',
                '18:00',
            ],
            'grid': '0.5/0.5',
            'format': 'netcdf',
        },
        f'{DATADIR}/Wind/{year}.nc')

2023-01-24 12:01:25,110 INFO Welcome to the CDS
2023-01-24 12:01:25,112 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-01-24 12:01:25,154 INFO Request is queued
2023-01-24 12:01:26,180 INFO Request is running
2023-01-24 12:09:43,558 INFO Request is completed
2023-01-24 12:09:43,559 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data4/adaptor.mars.internal-1674558412.3966312-531-3-49f04b2a-2fb7-4532-b259-c6de4579192c.nc to /scratch2/hugo/ERA5/Wind/1960.nc (2.8G)
2023-01-24 12:11:00,552 INFO Download rate 37.7M/s  
2023-01-24 12:11:00,727 INFO Welcome to the CDS
2023-01-24 12:11:00,729 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-01-24 12:11:00,783 INFO Request is queued
2023-01-24 12:11:01,807 INFO Request is running
2023-01-24 12:21:19,315 INFO Request is completed
2023-01-24 12:21:19,317 INFO Downloading https:

In [ ]:
c = cdsapi.Client()
for year in np.unique(DATERANGEPL.year)[:]:
    c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'variable': [
                'u_component_of_wind',
                'v_component_of_wind',
            ],
            'pressure_level': [
                '300'
            ],
            'year': str(year),
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '06:00', '12:00',
                '18:00',
            ],
            'grid': '0.5/0.5',
            'format': 'netcdf',
        },
        f'{DATADIR}/Wind/{year}.nc')

In [ ]:
import cdsapi
import shutil

c = cdsapi.Client()
months = [str(i) for i in range(1, 13)]
years = [str(i) for i in range(1997, 2023)]
varname = "geopotential"
grid = "1.0/1.0"
plevel = "500"
for year in years:
    for month in months:
        if (year, month) in [("1997", f"{i}") for i in range(1, 12)]:
            continue 
        print("year", year, "month", month)
        result = c.service(
            "tool.toolbox.orchestrator.workflow",
            
            params={
                "realm": "user-apps",
                "project": 'app-c3s-daily-era5-statistics',
                "version": "master",
                "kwargs": {
                    "dataset": "reanalysis-era5-pressure-levels",
                    "product_type": "reanalysis",
                    "variable": varname,
                    "statistic": "daily_mean",
                    "year": year,
                    "month" : month,
                    "time_zone": "UTC+00:0",
                    "frequency": "1-hourly",
                    "grid": grid,
                    "area": {"lat": [15, 80], "lon": [-180, 179]},
                    "pressure_level": plevel,
                },
                "workflow_name": "application"
            }
        )
        c.download(result)
        
        filename = f"{DATADIR}/{varname}_{plevel}hPa_{year}{month}_{grid.split('/')[0]}deg.nc"
        shutil.move(result[0]["location"].split("/")[-1], filename)

In [1]:
from ecmwfapi import ECMWFDataServer
server = ECMWFDataServer()
server.retrieve({
    "class": "e4",
    "dataset": "era40",
    "date": "1957-12-01/to/2002-03-31",
    "levelist": "700/775/850/925",
    "levtype": "pl",
    "param": "131.128/132.128",
    "step": "0",
    "stream": "oper",
    "time": "00:00:00/06:00:00/12:00:00/18:00:00",
    "type": "an",
    "target": "/scratch2/hugo/ERA40/UandV.nc",
})

2023-01-04 12:13:45 ECMWF API python library 1.6.3
2023-01-04 12:13:45 ECMWF API at https://api.ecmwf.int/v1
2023-01-04 12:13:46 Welcome Hugo Banderier
2023-01-04 12:13:46 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web+API+FAQ or contact servicedesk@ecmwf.int
2023-01-04 12:13:46 Request submitted
2023-01-04 12:13:46 Request id: 63b55f6a906002fb28784b4d
2023-01-04 12:13:46 Request is submitted
2023-01-04 12:13:53 Request is active
2023-01-04 13:54:59 mars - INFO   - 20230104.125437 - Looking up FDB indexes: 0.048037 second elapsed, 0.042639 second cpu [ATOS FDB BC]
2023-01-04 13:54:59 mars - INFO   - 20230104.125437 - Calling mars on 'marser-core', local port is 36066
2023-01-04 13:54:59 mars - INFO   - 20230104.125438 - Server task is 559 [marser]
2023-01-04 13:54:59 mars - INFO   - 20230104.125438 - Request cost: 992 fields, 49.0978 Mbytes online, nodes: mvr006 [marser]
2023-01-04 13:54:59 mars - INFO   - 20230104.125438 - The efficiency of your requ